## One_direction_by_modes_tables

### Import libraries

In [1]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
from sqlalchemy import create_engine
from sqlalchemy import text
from shapely.geometry import LineString, Point
from shapely.ops import unary_union

#### Create the connection with the DB

In [2]:
# Step 1: Create the connection with the DB
engine = create_engine("postgresql://urbaninfo:@cirrus.ita.chalmers.se/se_tuptp")
conn = engine.connect()

In [3]:
# Step 2: Create a new schema when it is necessary
#schema = '''CREATE SCHEMA name_schema'''
#conn.execute(schema)

### Nodes

In [4]:
# Import Data
# Step 1: Import Data
nodes_pt_1_query = text('SELECT * FROM pt_1_mdirections_mstops_mroutes.nodes_pt_1')
nodes_pt_1 = gpd.read_postgis(nodes_pt_1_query, engine, geom_col='geometry')

In [5]:
# Preprocess the data.
# Step 1: Copy the original DataFrame
nodes_pt_3_id = nodes_pt_1.copy()

# Step 2: Create the 'node_id' column by concatenating 'stop_name', 'place_id', and 'route_id' with underscores
nodes_pt_3_id ['node_id'] = nodes_pt_3_id['stop_name'].astype(str) +'_'\
                            + nodes_pt_3_id['place_id'].astype(str) +'_'\
                            + nodes_pt_3_id['mode'].astype(str)

# Step 3: Define aggregation functions for each column
agg_func_node={
    'route_id': list,
    'route_short_name': list,
    'stop_id':list,
    'stop_name':'first',
    'place_id':'first',
    'mode':'first',
    'direction_id':list,
    'osmid':list,
    'geometry': lambda g: Point(unary_union(g).centroid.x, unary_union(g).centroid.y)
}

nodes_pt_3_grouped = nodes_pt_3_id.groupby('node_id').agg(agg_func_node).reset_index()

In [6]:
# Filter and organize the data
# Step 4: Select and clean the final columns
nodes_pt_3 = nodes_pt_3_grouped [['node_id', 'stop_name', 'place_id', 'mode',
                                  'stop_id', 'route_id', 'route_short_name',
                                  'direction_id', 'osmid', 'geometry']].dropna().drop_duplicates('node_id')

# Step 5 (optional): Display the head of the DataFrame
nodes_pt_3.head()

,node_id,stop_name,place_id,mode,stop_id,route_id,route_short_name,direction_id,osmid,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,bus_service,"[9022014082785001, 9022014082785001, 902201408...","[9011014200300000, 9011014200500000, 901101420...","[3, 5, 8, 1, 3, 5, 8, 1]","[0, 1, 0, 0, 1, 0, 1, 1]","[87793055.0, 87793055.0, 87793055.0, 87793055....",POINT (378291.977 6399668.440)
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,bus_service,"[9022014015161001, 9022014015161001, 902201401...","[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 932, 331, 334, 4]","[0, 0, 1, 1, 1, 1, 0]","[1021141732.0, 1021141732.0, 1021141732.0, 102...",POINT (313191.955 6442732.480)
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,bus_service,[9022014046020001],[9011014658100000],[581],[1],[173516827.0],POINT (371000.976 6448532.958)
3,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,307959538,communal_taxi_service,"[9022014046020001, 9022014046020002]","[9011014366300000, 9011014366300000]","[663, 663]","[1, 0]","[173516827.0, 835571452.0]",POINT (371017.481 6448504.455)
4,Acklinga Solbacken_307927983_bus_service,Acklinga Solbacken,307927983,bus_service,[9022014042270002],[9011014332600000],[326],[1],[1008032531.0],POINT (434378.988 6451901.915)


#### export to the DB

In [7]:
# Step 6 (optional): Set geometry and export to the DB
nodes_pt_3 = gpd.GeoDataFrame(nodes_pt_3, geometry='geometry', crs='3006')
nodes_pt_3.to_postgis('nodes_pt_3', engine, schema='pt_3_odirection_mstops_bymode', if_exists ='replace')

### Links

In [8]:
# Import data
#Step 1: Import Data
links_pt_1_query = text('SELECT * FROM pt_1_mdirections_mstops_mroutes.links_pt_1_wide')
links_pt_1 = gpd.read_postgis(links_pt_1_query, engine, geom_col='geometry')

In [9]:
# Preprocess the Data
# Step 1: Merge nodes and links in order to get the right combination of data
links_pt_3_source_merge = pd.merge(
    nodes_pt_3[['node_id', 'stop_name', 'place_id', 'mode', 'direction_id', 'geometry']],
    links_pt_1[['route_id', 'route_short_name', 'stop_id_source', 'stop_id_target',
                'stop_name_source', 'stop_name_target', 'place_id_source', 'place_id_target',
                'mode','time_distance'] + [col for col in links_pt_1.columns if col.startswith('freq_')]],
    left_on=['mode', 'stop_name', 'place_id'],
    right_on=['mode', 'stop_name_source', 'place_id_source'], how='inner'
)

links_pt_3_target_merge  = pd.merge(
    nodes_pt_3[['node_id', 'stop_name', 'place_id', 'mode', 'direction_id', 'geometry']],
    links_pt_3_source_merge[['node_id', 'stop_id_source', 'stop_id_target', 'stop_name_source',
                             'stop_name_target', 'place_id_source', 'place_id_target',
                             'route_id', 'route_short_name', 'mode', 'time_distance',
                             'geometry'] + [col for col in links_pt_3_source_merge.columns if col.startswith('freq_')]],
    left_on=['mode','stop_name','place_id'],
    right_on=['mode','stop_name_target','place_id_target'],
    how='inner'
).rename(columns={
    'node_id_x':'node_id_target', 'node_id_y':'node_id_source',
    'geometry_x':'geometry_target', 'geometry_y':'geometry_source'
})

# Step 2: Select specific columns and remove duplicates
links_pt_3_l1 = links_pt_3_target_merge [['node_id_source', 'node_id_target', 'time_distance',
                                              'stop_name_source', 'stop_name_target', 'place_id_source',
                                              'place_id_target', 'stop_id_source', 'stop_id_target',
                                              'route_id', 'route_short_name', 'mode', 'direction_id',
                                              'geometry_source','geometry_target']
                                             + [col for col in links_pt_3_target_merge.columns if col.startswith('freq_')]]

# Step 3 (optional): Display the results
links_pt_3_l1.head()

,node_id_source,node_id_target,time_distance,stop_name_source,stop_name_target,place_id_source,place_id_target,stop_id_source,stop_id_target,route_id,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,129.0,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,9022014082785002,9011014200100000,...,0.0,0.0,0.0,2842.105263,1800.000000,2571.428571,1800.000000,2454.545455,4909.090909,0.0
1,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,114.0,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,9022014082785002,9011014200300000,...,0.0,0.0,0.0,9000.000000,9671.641791,9000.000000,9000.000000,9671.641791,54000.000000,0.0
2,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,117.0,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,9022014082785002,9011014200500000,...,0.0,0.0,0.0,8415.584416,5102.362205,8415.584416,4729.927007,6750.000000,9000.000000,0.0
3,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,117.0,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,9022014082785002,9011014200800000,...,0.0,0.0,0.0,6000.000000,3410.526316,4984.615385,3100.478469,5634.782609,9000.000000,0.0
4,Södra Älvsborgs sjukhus_307955489_bus_service,7:e Villagatan_307955489_bus_service,98.0,Södra Älvsborgs sjukhus,7:e Villagatan,307955489,307955489,9022014082726001,9022014082785001,9011014200100000,...,0.0,0.0,0.0,2781.115880,1825.352113,2623.481781,1800.000000,2347.826087,6000.000000,0.0


In [10]:
# Create the links of the one direction model
# Step 4: Aggregate Data
links_pt_3_l1['geometry'] = links_pt_3_l1.apply(lambda row: LineString([Point(row['geometry_source']), Point(row['geometry_target'])]), axis=1)
links_pt_3_l1['symmetric_pairs'] = links_pt_3_l1[['node_id_source', 'node_id_target']].apply(lambda x: '-'.join(sorted(x)), axis=1)

# Step 5: Define aggregation functions for various columns
agg_func_link = {
    'node_id_source': 'first',
    'node_id_target': 'first',
    'route_id': list,
    'route_short_name':list,
    'mode': 'first',
    'stop_id_source':'first',
    'stop_id_target':'first',
    'stop_name_source':'first',
    'stop_name_target':'first',
    'place_id_source':'first',
    'place_id_target':'first',
    'time_distance':'min',
    'direction_id':'first',
    'geometry': 'first'
}

freq_cols = [col for col in links_pt_3_l1.columns if col.startswith('freq_')]
agg_func_link.update({col: 'min' for col in freq_cols})

links_pt_3_l2 = links_pt_3_l1.groupby(['symmetric_pairs']).agg(agg_func_link).reset_index()

# Step 6 (optional): Display the results
links_pt_3_l2.head()

C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,symmetric_pairs,node_id_source,node_id_target,route_id,route_short_name,mode,stop_id_source,stop_id_target,stop_name_source,stop_name_target,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_bus_service-Bäckängss...,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,"[9011014200100000, 9011014200300000, 901101420...","[1, 3, 5, 8, 1, 3, 5, 8]",bus_service,9022014082676002,9022014082785002,Bäckängsskolan,7:e Villagatan,...,0.0,0.0,0.0,2842.105263,1800.000000,2571.428571,1800.0,2347.826087,0.0,0.0
1,7:e Villagatan_307955489_bus_service-Södra Älv...,Södra Älvsborgs sjukhus_307955489_bus_service,7:e Villagatan_307955489_bus_service,"[9011014200100000, 9011014200300000, 901101420...","[1, 3, 5, 8, 1, 3, 5, 8]",bus_service,9022014082726001,9022014082785001,Södra Älvsborgs sjukhus,7:e Villagatan,...,0.0,0.0,0.0,2757.446809,1825.352113,2623.481781,1800.0,2347.826087,0.0,0.0
2,AGA Vattenfall_307980814_bus_service-Borealis ...,Borealis kracker_307980814_bus_service,AGA Vattenfall_307980814_bus_service,"[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 331, 334, 4, 932]",bus_service,9022014015013002,9022014015161002,Borealis kracker,AGA Vattenfall,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
3,AGA Vattenfall_307980814_bus_service-Jordhamma...,Jordhammar_307980814_bus_service,AGA Vattenfall_307980814_bus_service,"[9011014633100000, 9011014633400000, 901101463...","[331, 334, 4, 932, 331, 334, 4]",bus_service,9022014015110001,9022014015161001,Jordhammar,AGA Vattenfall,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
4,Abrahamstorp_307959538_bus_service-Badet_30795...,Badet_307959538_bus_service,Abrahamstorp_307959538_bus_service,[9011014658100000],[581],bus_service,9022014046021001,9022014046020001,Badet,Abrahamstorp,...,0.0,0.0,0.0,54000.000000,27000.000000,27000.000000,27000.0,27000.000000,0.0,0.0


In [12]:
# Filter and organize the final table
# Step 7: Rename columns for clarity
links_pt_3_l2.rename(
    columns={'node_id_source': 'source', 'node_id_target': 'target'},
    inplace=True
)

# Step 8: Select desired columns for the final DataFrame
links_pt_3 = links_pt_3_l2 [['source', 'target', 'time_distance', 'route_id',
                             'stop_name_source', 'place_id_source','stop_name_target',
                             'place_id_target', 'stop_id_source', 'stop_id_target',
                             'route_short_name', 'mode', 'direction_id',
                             'geometry'] + [col for col in links_pt_3_l2.columns if col.startswith('freq_')]]

# Step 9: Filter out rows where 'source' is equal to 'target'
links_pt_3 = links_pt_3[links_pt_3['source'] != links_pt_3['target']]

# Step 10 (optional): Display the results
links_pt_3.head()

,source,target,time_distance,route_id,stop_name_source,place_id_source,stop_name_target,place_id_target,stop_id_source,stop_id_target,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,109.0,"[9011014200100000, 9011014200300000, 901101420...",Bäckängsskolan,307955489,7:e Villagatan,307955489,9022014082676002,9022014082785002,...,0.0,0.0,0.0,2842.105263,1800.000000,2571.428571,1800.0,2347.826087,0.0,0.0
1,Södra Älvsborgs sjukhus_307955489_bus_service,7:e Villagatan_307955489_bus_service,86.0,"[9011014200100000, 9011014200300000, 901101420...",Södra Älvsborgs sjukhus,307955489,7:e Villagatan,307955489,9022014082726001,9022014082785001,...,0.0,0.0,0.0,2757.446809,1825.352113,2623.481781,1800.0,2347.826087,0.0,0.0
2,Borealis kracker_307980814_bus_service,AGA Vattenfall_307980814_bus_service,27.0,"[9011014633100000, 9011014633400000, 901101463...",Borealis kracker,307980814,AGA Vattenfall,307980814,9022014015013002,9022014015161002,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
3,Jordhammar_307980814_bus_service,AGA Vattenfall_307980814_bus_service,97.0,"[9011014633100000, 9011014633400000, 901101463...",Jordhammar,307980814,AGA Vattenfall,307980814,9022014015110001,9022014015161001,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
4,Badet_307959538_bus_service,Abrahamstorp_307959538_bus_service,82.0,[9011014658100000],Badet,307959538,Abrahamstorp,307959538,9022014046021001,9022014046020001,...,0.0,0.0,0.0,54000.000000,27000.000000,27000.000000,27000.0,27000.000000,0.0,0.0


In [14]:
# Step 11 (optional): create the GeoDataFrame and export to the DB
links_pt_3 = gpd.GeoDataFrame(links_pt_3, geometry='geometry', crs='3006')
links_pt_3.to_postgis('links_pt_3', engine, schema='pt_3_odirection_mstops_bymode', if_exists ='replace')

### Transfers

#### Same stop

In [15]:
# Preprocess the Data
# Step 1: Create a copy of the nodes table
nodes_transfer_same_stop = nodes_pt_3[['node_id', 'stop_name', 'place_id', 'geometry']].copy()

#Step 2 (optional): display the results
nodes_transfer_same_stop.head()

,node_id,stop_name,place_id,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480)
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958)
3,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,307959538,POINT (371017.481 6448504.455)
4,Acklinga Solbacken_307927983_bus_service,Acklinga Solbacken,307927983,POINT (434378.988 6451901.915)


In [16]:
# Step 3: Merge the stops in order to create the transfers
# Source
stop_routes_source = nodes_transfer_same_stop.rename(columns={'node_id': 'source', 'stop_name': 'target'}).drop_duplicates()

# Target
stop_routes_target = pd.merge(nodes_transfer_same_stop,
                              nodes_pt_3[['node_id']],
                              on='node_id',
                              how='inner').rename(columns={'stop_name': 'source', 'node_id': 'target'}).drop_duplicates()

# Step 4: Create the transfers same stop table
transfers_same_stop = pd.merge(stop_routes_source, stop_routes_target, how='outer')

#Step 5 (optional): Display the results
transfers_same_stop.head()

,source,target,place_id,geometry
0,7:e Villagatan,7:e Villagatan_307955489_bus_service,307955489,POINT (378291.977 6399668.440)
1,7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
2,AGA Vattenfall,AGA Vattenfall_307980814_bus_service,307980814,POINT (313191.955 6442732.480)
3,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480)
4,Abrahamstorp,Abrahamstorp_307959538_bus_service,307959538,POINT (371000.976 6448532.958)


In [17]:
# Step 6: Merge the table with the frequency of the links table
transfers_same_stop = pd.merge(transfers_same_stop,
                               links_pt_3[['target'] + [col for col in links_pt_3.columns if col.startswith('freq_')]],
                               on='target',
                               how='outer').drop_duplicates(subset=('source', 'target'))
transfers_same_stop = transfers_same_stop[['source', 'target', 'geometry'] + [col for col in transfers_same_stop.columns if col.startswith('freq_')]]

#Step 7 (optional): Display the results
transfers_same_stop.head()

,source,target,geometry,freq_monday_morning,freq_tuesday_morning,freq_wednesday_morning,freq_thursday_morning,freq_friday_morning,freq_saturday_morning,freq_sunday_morning,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7:e Villagatan,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440),568.421053,568.421053,568.421053,568.421053,568.421053,635.294118,635.294118,...,0.0,0.0,0.0,2842.105263,1800.0,2571.428571,1800.0,2347.826087,0.0,0.0
3,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,POINT (313191.955 6442732.480),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGA Vattenfall,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
6,Abrahamstorp_307959538_bus_service,Abrahamstorp,POINT (371000.976 6448532.958),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Step 8: Replace 0 with NaN and vice versa in frequency columns
freq_columns = [col for col in transfers_same_stop.columns if col.startswith('freq_')]
transfers_same_stop[freq_columns] = transfers_same_stop[freq_columns].replace({0: np.nan, pd.NA: 0})

#Step 9 (optional): Display the results
transfers_same_stop.head()

,source,target,geometry,freq_monday_morning,freq_tuesday_morning,freq_wednesday_morning,freq_thursday_morning,freq_friday_morning,freq_saturday_morning,freq_sunday_morning,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7:e Villagatan,7:e Villagatan_307955489_bus_service,POINT (378291.977 6399668.440),568.421053,568.421053,568.421053,568.421053,568.421053,635.294118,635.294118,...,NaN,NaN,NaN,2842.105263,1800.0,2571.428571,1800.0,2347.826087,NaN,NaN
3,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,POINT (313191.955 6442732.480),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AGA Vattenfall,AGA Vattenfall_307980814_bus_service,POINT (313191.955 6442732.480),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Abrahamstorp_307959538_bus_service,Abrahamstorp,POINT (371000.976 6448532.958),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Step 10 (optional): Set geometry and export to the DB
transfers_same_stop = gpd.GeoDataFrame(transfers_same_stop, geometry='geometry', crs='3006')
transfers_same_stop.to_postgis('transfers_same_stop_pt_3', engine,
                               schema='pt_3_odirection_mstops_bymode',if_exists ='replace')

#### Different stops

In [20]:
#Preprocess the Data
# Step 1: Create a copy of the nodes table
nodes_transfer_diff_stop = nodes_pt_3[['node_id', 'stop_name', 'place_id', 'geometry']].copy()

# Step 2 (optional): Display the results
nodes_transfer_diff_stop.head()

,node_id,stop_name,place_id,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480)
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958)
3,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,307959538,POINT (371017.481 6448504.455)
4,Acklinga Solbacken_307927983_bus_service,Acklinga Solbacken,307927983,POINT (434378.988 6451901.915)


In [21]:
# Create the buffers
# Step 3: Create a buffer column and set it as geometry
nodes_transfer_diff_stop['buffer'] = nodes_transfer_diff_stop['geometry'].buffer(85)
nodes_transfer_diff_stop = nodes_transfer_diff_stop.set_geometry('buffer',
                                                                 crs='EPSG:3006')

# Step 4: found the intersections among the buffers
nodes_transfer_diff_stop = gpd.overlay(nodes_transfer_diff_stop,
                                       nodes_transfer_diff_stop,
                                       how='intersection').rename(columns={'node_id_1':'source',
                                                                           'node_id_2':'target',
                                                                           'stop_name_1':'stop_name_source',
                                                                           'stop_name_2':'stop_name_target',
                                                                           'geometry_1': 'geometry_source',
                                                                           'geometry_2': 'geometry_target'}).dropna(subset=['source'])
# Step 5 (optional): Display the results
nodes_transfer_diff_stop.head()

,source,stop_name_source,place_id_1,geometry_source,target,stop_name_target,place_id_2,geometry_target,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"POLYGON ((378376.568 6399660.109, 378375.344 6..."
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480),AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480),"POLYGON ((313276.546 6442724.148, 313275.322 6..."
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958),Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958),"POLYGON ((371085.567 6448524.626, 371084.343 6..."
3,Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958),Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,307959538,POINT (371017.481 6448504.455),"POLYGON ((371085.567 6448524.626, 371084.343 6..."
4,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,307959538,POINT (371017.481 6448504.455),Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958),"POLYGON ((370936.141 6448479.781, 370934.114 6..."


In [22]:
#Create the geometry
# Step 6: Create linstrings between the intersections
nodes_transfer_diff_stop['geometry'] = nodes_transfer_diff_stop.apply(
    lambda row: LineString([[row.geometry_source.x,  row.geometry_source.y],
                            [row.geometry_target.x,row.geometry_target.y]])
    if row.geometry_target is not None else None,
    axis=1
)
# Step 7: Set the length of the linestring
nodes_transfer_diff_stop['length'] = nodes_transfer_diff_stop['geometry'].length
nodes_transfer_diff_stop['time_distance'] = nodes_transfer_diff_stop['length'] / 1.2

# Step 8 (optional): Display the results
nodes_transfer_diff_stop.head()

,source,stop_name_source,place_id_1,geometry_source,target,stop_name_target,place_id_2,geometry_target,geometry,length,time_distance
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_bus_service,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"LINESTRING (378291.977 6399668.440, 378291.977...",0.000000,0.000000
1,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480),AGA Vattenfall_307980814_bus_service,AGA Vattenfall,307980814,POINT (313191.955 6442732.480),"LINESTRING (313191.955 6442732.480, 313191.955...",0.000000,0.000000
2,Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958),Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958),"LINESTRING (371000.976 6448532.958, 371000.976...",0.000000,0.000000
3,Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958),Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,307959538,POINT (371017.481 6448504.455),"LINESTRING (371000.976 6448532.958, 371017.481...",32.936222,27.446852
4,Abrahamstorp_307959538_communal_taxi_service,Abrahamstorp,307959538,POINT (371017.481 6448504.455),Abrahamstorp_307959538_bus_service,Abrahamstorp,307959538,POINT (371000.976 6448532.958),"LINESTRING (371017.481 6448504.455, 371000.976...",32.936222,27.446852


In [23]:
#Filter and organize the tables
# Step 9: Filter the transfers between different stops
transfers_diff_stop = nodes_transfer_diff_stop[nodes_transfer_diff_stop['length'] > 0]

# Step 10: Drop the duplicates of symmetric pairs
transfers_diff_stop ['symmetric_pairs'] = transfers_diff_stop[['source',
                                                               'target']].apply(lambda x: '-'.join(sorted(x)),
                                                                                axis=1)
transfers_diff_stop = transfers_diff_stop.drop_duplicates(subset = ['symmetric_pairs'])

# Step 11: Filter the necessary columns
transfers_diff_stop = transfers_diff_stop [['stop_name_source',
                                            'stop_name_target',
                                            'geometry',
                                            'length',
                                            'time_distance']].rename(columns={'stop_name_source':'source',
                                                                              'stop_name_target':'target'}).drop_duplicates()
# Step 12 (optional): Display the results
transfers_diff_stop.head()

C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,source,target,geometry,length,time_distance
3,Abrahamstorp,Abrahamstorp,"LINESTRING (371000.976 6448532.958, 371017.481...",32.936222,27.446852
9,Acklinga kyrka,Acklinga kyrka,"LINESTRING (434071.965 6451013.922, 434077.950...",6.070173,5.058478
13,Acklinga,Acklinga,"LINESTRING (434427.979 6451013.415, 434427.976...",5.512155,4.593462
26,Agerhus,Jordbron,"LINESTRING (324621.469 6533086.938, 324604.463...",125.603619,104.669683
35,Agnebergsskolan,Agnebergsskolan,"LINESTRING (321088.468 6471557.963, 321077.949...",28.982209,24.151840


In [24]:
#export to the db
transfers_diff_stop = gpd.GeoDataFrame(transfers_diff_stop, geometry='geometry', crs='3006')
transfers_diff_stop.to_postgis('transfers_diff_stop_pt_3', engine,
                               schema='pt_3_odirection_mstops_bymode', if_exists ='replace')